# Experiments for "DBSpan: Density-Based Spanner for Clustering Complex Data, With an Application to Persistence Diagrams"

To get started, we will do a little bit of path hackery to import the library.  There are better ways to do this, but for now, this is okay.

In [ ]:
import random
import sys
import os
import math

libpath = os.path.abspath('..')
sys.path.insert(0, libpath)

import dbspan

We will do a bunch of comparisions of DBScan and DBSpan, so it will help if we have some code for making the algos

In [ ]:
dgm_metric = dbspan.topology.DiagramMetric()

def dgm1_metric(dgm1, dgm2):
    return dgm_metric.bottleneck(dgm1[1], dgm2[1])

def make_algos(eps, min_samples, delta):
    algo_dbscan = dbspan.cluster.DBSCAN(metric=dgm1_metric, eps=eps, min_samples=min_samples)
    algo_dbspan = dbspan.cluster.DBSpan(metric=dgm1_metric, eps=eps, min_samples=min_samples, delta=delta)

    return algo_dbscan, algo_dbspan

## Experiment 1: Point clouds of things in 4D

First, we will create some tools for creating dgms of things in 4D.  Each function will produce a diagram from a rips filtration of a "noisy" sphere, torus, or swiss roll.

In [ ]:
data_factory = dbspan.topology.PointSetDataFactory(seed=72330)
dgm_factory = dbspan.topology.DiagramFactory()

rng = random.Random(43081)

def make_sphere_dgm():
    points = data_factory.make_sphere(dim=4, num_points=100, noise=.1)
    return dgm_factory.make_from_point_set(points)

def make_torus_dgm():
    points = data_factory.make_torus(dim=4, num_points=100, noise=.1)
    return dgm_factory.make_from_point_set(points)

def make_swiss_roll_dgm():
    points = data_factory.make_swiss_roll(dim=4, num_points=100, noise=.1)
    return dgm_factory.make_from_point_set(points)

Now that we have some functions for creating data, let's create our data set that we will cluster:

In [ ]:
num_dgms1 = 30
dgms1 = [make_sphere_dgm() for _ in range(num_dgms1)] \
    + [make_torus_dgm() for _ in range(num_dgms1)] \
    + [make_swiss_roll_dgm() for _ in range(math.floor(num_dgms1/2) - 1)]

And we are off!  Let's do some experiments

In [ ]:
import time
import pandas as pd
from sklearn import metrics

def make_cache(true_labels, dbscan_time):
    return {
        'true_labels': true_labels,
        'dbscan_time': dbscan_time,
    }  
    
def run_experiment1(df, data, delta, eps, min_samples, cache=None):
    
    def add_row(df, delta, num_edges, max_edges, rand_index, dbscan_time, dbspan_time):
        data = [delta, rand_index, num_edges, max_edges, num_edges/max_edges, dbscan_time, dbspan_time, dbscan_time/dbspan_time]
        cols=['$\delta$', 'Rand index', 'Num Edges', 'maxEdges', '\% Possible Edges', 'T_DBSCAN', 'DBSpan time (sec)', 'Speedup']
        line = pd.DataFrame([data,], columns=cols)
        return pd.concat([df, line])
   
    # create the algos
    algo_dbscan, algo_dbspan = make_algos(eps=eps, min_samples=min_samples, delta=delta)

    # run the algos
    t0 = time.perf_counter()
    true_labels = cache['true_labels'] if cache else algo_dbscan.fit(data)
    t1 = time.perf_counter()
    dbspan_labels, dbg_data= algo_dbspan.fit(data, dbg=True)
    t2 = time.perf_counter()
    
    # pull out the dbg data for the analysis
    spanner = dbg_data['neighborhood'].spanner
    
    # prepare data for row
    rand_index = metrics.adjusted_rand_score(true_labels, dbspan_labels)
    dbscan_time = cache['dbscan_time'] if cache else t1 - t0
    dbspan_time = t2 - t1
    num_edges = spanner.number_of_edges()
    n = spanner.number_of_nodes()
    max_edges = n * (n-1) / 2
    
    # return row
    return add_row(df, delta, num_edges, max_edges, rand_index, dbscan_time, dbspan_time), \
        make_cache(true_labels, dbscan_time)

In [ ]:
results1 = None
cache1 = None
eps1 = .3
min_samples1 = 15
for delta in [.1, 1, 10, 50, 100, 500, 1000]:
    results1, cache1 = run_experiment1(results1, dgms1, delta, eps1, min_samples1, cache=cache1)
    print(results1)

print(results1.style.to_latex())

# Experiment 2

First, we will load in the data.  Since it is pretty small when gzipped, we keep it in the repo.  Let's unzip it

In [ ]:
!mkdir -p data
!curl -J -L https://osf.io/4nwe9/download | tar xz --directory data
!echo "done!"

In [ ]:
import os
import numpy as np

exp2_data_dir = './data/experiment2/'
exp2_files = os.listdir(exp2_data_dir)

def read_file(name):
    file_name = os.path.join(exp2_data_dir, name)
    return [None, np.genfromtxt(file_name, delimiter=',')]

dgms2 = [ read_file(name) for name in exp2_files ]
print("Read in {} dgms".format(len(dgms2)))

Next, let's look at the distribution of the number of points in the diagrams.  Personally, I don't know much about this data set, but MSU local expert (and super well guy that gave me these diagrams) Jordan Schupbach said that number of diagram points could be a useful to look at.  So let's look at the distribution of of the number of points.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

npts = np.array([dgm[1].shape[0] for dgm in dgms2])
plt.hist(npts)

So, it looks like we have a bunch of smallish dgms and a few very large diagrams.  To get a good distribution of diagram sizes, we will create three levels each containing the same number of diagrams.

In [ ]:
dgms_by_level = [
    np.argwhere(npts < 150),
    np.argwhere((150 <= npts) & (npts < 200)),
    np.argwhere(200 <= npts),
]

def make_data2(dgms_per_level):
    rng = np.random.default_rng(seed=0)
    return [
        dgms2[idx]
        for level in dgms_by_level
        for idx in rng.choice(level.transpose()[0], size=dgms_per_level, replace=False)
    ]

small_data2 = make_data2(dgms_per_level=10)

Next, let's look at the pairwise distance matrix of a very small dataset.  (Note that the code below takes quite a while to run and stare at to pick some constants.  Uncomment to check out the distances).

In [ ]:
# for i in range(len(small_data2)):
#     for j in range(i+1, len(small_data2)):
#         print(i, j, dgm1_metric(small_data2[i], small_data2[j]))

So, it looks like we have a bunch of close diagrams all with distance less than 10 and then some outliers.  So, let's cluster our points using and epsilon of 10 and see how it goes.

In [ ]:
eps2 = 10
min_samples2 = 5

algo_dbscan = dbspan.cluster.DBSCAN(metric=dgm1_metric, eps=eps2, min_samples=min_samples2)

# run the algos
t0 = time.perf_counter()
true_labels = algo_dbscan.fit(small_data2)
t1 = time.perf_counter()
dbscan_time = t1-t0
cache2 = make_cache(true_labels, dbscan_time)
print(cache2)

So, as we expected, we have a one cluster of diagrams and some noise.  Now, let's run the same experiments that we ran before to pick a good value for delta.

In [ ]:
results2 = None
for delta in [.1, 1, 2, 3, 4, 5, 6]:
    results2, cache2 = run_experiment1(results2, small_data2, delta, eps2, min_samples2, cache=cache2)
    print(results2)
print(results2.style.to_latex())

Excellent!!  So what did we learn, well, we can get a pretty good speed up and maintain pretty good accuracy if we pick delta just before the rand index dropps off.  Now let's see how large of a data set we can process.

In [ ]:
delta2 = 1

import logging
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')

def run_experiment2(df, num_per_level, delta, eps, min_samples):
  
    def add_row(df, num_dgms, num_edges, max_edges, rand_index, dbscan_time, dbspan_time):
        data = [num_dgms, rand_index, num_edges, max_edges, num_edges/max_edges, dbscan_time, dbspan_time, dbscan_time/dbspan_time]
        cols=['Num Dgms', 'Rand index', 'Num Edges', 'maxEdges', '\% Possible Edges', 'T_DBSCAN', 'DBSpan time (sec)', 'Speedup']
        line = pd.DataFrame([data,], columns=cols)
        return pd.concat([df, line])
  
    d2 = make_data2(dgms_per_level=num_per_level)
    
    algo_dbscan, algo_dbspan = make_algos(eps=eps, min_samples=min_samples, delta=delta)

    # run the algos
    t0 = time.perf_counter()
    dbspan_labels, dbg_data= algo_dbspan.fit(d2, dbg=True)
    dbspan_time = time.perf_counter() - t0

    # prep dbspan data
    spanner = dbg_data['neighborhood'].spanner    
    num_edges = spanner.number_of_edges()
    n = spanner.number_of_nodes()
    max_edges = n * (n-1) / 2
    
    logging.info((n, dbspan_time, num_edges, max_edges))
    
    # run dbscan
    t1 = time.perf_counter()
    true_labels = algo_dbscan.fit(d2)
    dbscan_time = time.perf_counter() - t1
    
    # prepare data for row
    rand_index = metrics.adjusted_rand_score(true_labels, dbspan_labels)   

    # return row
    return add_row(df, n, num_edges, max_edges, rand_index, dbscan_time, dbspan_time)
    
results2b = None
eps2 = 10
min_samples2 = 5
for i in [10, 15, 20, 25, 30, 35, 40]:
    results2b = run_experiment2(results2b, i, delta2, eps2, min_samples2)
    print(results2b)

print(results2b.style.to_latex())